# API REQUESTS

In [48]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv, find_dotenv
import time

In [77]:
load_dotenv() # moves all variables from .env file to notebook memory.

my_key = os.getenv("API_KEY")


## Hotels in Lisbon:

In [50]:
# Hotels in Lisbon

url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={my_key}&searchQuery=Lisbon&category=hotels&language=en"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{
  "data": [
    {
      "location_id": "276952",
      "name": "Corinthia Lisbon",
      "address_obj": {
        "street1": "Avenida Columbano Bordalo Pinheiro, 105",
        "street2": "",
        "city": "Lisbon",
        "country": "Portugal",
        "postalcode": "1099-031",
        "address_string": "Avenida Columbano Bordalo Pinheiro, 105, Lisbon 1099-031 Portugal"
      }
    },
    {
      "location_id": "548451",
      "name": "Bairro Alto Hotel",
      "address_obj": {
        "street1": "Praca Luis de Camoes 2",
        "street2": "",
        "city": "Lisbon",
        "country": "Portugal",
        "postalcode": "1200-243",
        "address_string": "Praca Luis de Camoes 2, Lisbon 1200-243 Portugal"
      }
    },
    {
      "location_id": "12659702",
      "name": "Corpo Santo Lisbon Historical Hotel",
      "address_obj": {
        "street1": "Largo do Corpo Santo, 25",
        "city": "Lisbon",
        "country": "Portugal",
        "postalcode": "1200-129",
        

In [ ]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

SEARCH_URL = "https://api.content.tripadvisor.com/api/v1/location/search"
DETAILS_URL = "https://api.content.tripadvisor.com/api/v1/location/{}/details"

headers = {"accept": "application/json"}

queries = [
    "Lisbon hotel",
    "Lisbon hotels",
    "Lisbon accommodation",
    "Lisbon boutique hotel",
    "Lisbon luxury hotel",
    "Lisbon guesthouse",
    "Lisbon bed and breakfast",
]

all_hotels = []


for q in queries:
    url = (
        f"{SEARCH_URL}?key={API_KEY}"
        f"&searchQuery={q}"
        f"&category=hotels"
        f"&language=en"
    )

    response = requests.get(url, headers=headers)
    data = response.json().get("data", [])
    all_hotels.extend(data)

# Convert to unique hotel list
df = pd.DataFrame(all_hotels).drop_duplicates(subset=["location_id"])
df.reset_index(drop=True, inplace=True)

print(f"\n Total unique hotels collected: {len(df)}")
print(" Fetching rating details for each hotel...\n")

# Fetch ratings:
hotel_rows = []

for i, hotel in df.iterrows():
    loc_id = hotel["location_id"]
    detail_url = DETAILS_URL.format(loc_id)

    params = {"key": API_KEY, "language": "en", "currency": "EUR"}

    detail_res = requests.get(detail_url, headers=headers, params=params)
    if detail_res.status_code != 200:
        continue

    details = detail_res.json()

    hotel_rows.append({
        "location_id": loc_id,
        "name": details.get("name"),
        "rating": details.get("rating"),
        "num_reviews": details.get("num_reviews"),
        "ranking": details.get("ranking"),
        "price_level": details.get("price_level"),
        "street1": details.get("address_obj", {}).get("street1"),
        "city": details.get("address_obj", {}).get("city"),
        "country": details.get("address_obj", {}).get("country"),
        "postalcode": details.get("address_obj", {}).get("postalcode"),
        "address_string": details.get("address_obj", {}).get("address_string"),
    })


    time.sleep(0.2)  # avoid rate limits

# Final DataFrame
df_hotels = pd.DataFrame(hotel_rows)

# Save CSV
output_file = "lisbon_hotels.csv"
df_hotels.to_csv(output_file, index=False)


🔎 Collecting hotel list...

📌 Total unique hotels collected: 48
📥 Fetching rating details for each hotel...


🎉 DONE! Saved 48 hotels with ratings to lisbon_hotels.csv


- I got 48 hotels extracted

In [73]:
print(len(df_hotels))

48


In [76]:
df_hotels.head(10)

,location_id,name,rating,num_reviews,ranking,price_level,street1,city,country,postalcode,address_string
0,2292067,Czar Lisbon Hotel,3.7,947,None,$$,Avenida Almirante Reis 103,Lisbon,Portugal,1150-0020,"Avenida Almirante Reis 103, Lisbon 1150-0020 P..."
1,206890,Avani Avenida Liberdade Lisbon Hotel,4.3,796,None,$$$,Rua Julio Cesar Machado 7 9,Lisbon,Portugal,1250-135,"Rua Julio Cesar Machado 7 9, Lisbon 1250-135 P..."
2,11600027,Empire Lisbon Hotel,4.3,398,None,$$,Avenida Almirante Reis 130,Lisbon,Portugal,1150-023,"Avenida Almirante Reis 130, Lisbon 1150-023 Po..."
3,8842153,Palacio Do Governador - Lisbon Hotel & Spa,4.4,526,None,$$$,"Rua Bartolomeu Dias, 117",Lisbon,Portugal,1400-030,"Rua Bartolomeu Dias, 117, Lisbon 1400-030 Port..."
4,19780664,Aroeira Lisbon Hotel - Sea & Golf Resort,4.1,93,None,$$,Avenida Pinhal da Aroeira 1,Aroeira,Portugal,2820-112,Avenida Pinhal da Aroeira 1 Herdade da Aroeira...
5,553361,Dinya Lisbon Hotel & Lounge Bar,4.3,34,None,$$,Rua Ilha do Pico 3,Lisbon,Portugal,1000-169,"Rua Ilha do Pico 3, Lisbon 1000-169 Portugal"
6,781013,New Style Lisbon Hotel,3.6,97,None,$$,Avenida Almirante Reis 53,Lisbon,Portugal,1150-011,"Avenida Almirante Reis 53, Lisbon 1150-011 Por..."
7,12659702,Corpo Santo Lisbon Historical Hotel,4.9,4398,None,$$$$,"Largo do Corpo Santo, 25",Lisbon,Portugal,1200-129,"Largo do Corpo Santo, 25, Lisbon 1200-129 Port..."
8,13402042,Hotel Da Baixa,4.9,2867,None,$$$,Rua da Prata 231,Lisbon,Portugal,1100-417,"Rua da Prata 231, Lisbon 1100-417 Portugal"
9,195643,Hotel Avenida Palace,4.8,5025,None,$$$$,Rua 1 Dezembro 123,Lisbon,Portugal,1200-359,"Rua 1 Dezembro 123, Lisbon 1200-359 Portugal"


## Restaurants in Lisbon:

In [79]:
# Load API key
load_dotenv()
API_KEY = os.getenv("API_KEY")

SEARCH_URL = "https://api.content.tripadvisor.com/api/v1/location/search"
DETAILS_URL = "https://api.content.tripadvisor.com/api/v1/location/{}/details"

headers = {"accept": "application/json"}

# Expand the search to collect the maximum number of restaurants
queries = [
    "Lisbon restaurant",
    "restaurants Lisbon",
    "Lisbon food",
    "Lisbon dining",
    "Lisbon cafe",
    "Lisbon brunch",
    "Lisbon lunch",
    "Lisbon dinner",
    "Lisbon Michelin",
    "Lisbon traditional food",
    "Lisbon cheap eats",
]

all_restaurants = []

# Searching Restaurants:

for q in queries:
    url = (
        f"{SEARCH_URL}?key={API_KEY}"
        f"&searchQuery={q}"
        f"&category=restaurants"
        f"&language=en"
    )

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Error for '{q}' → {response.text}")
        continue

    data = response.json().get("data", [])
    all_restaurants.extend(data)

    print(f"'{q}' returned {len(data)} restaurants")


# Deduplicate by TripAdvisor Location ID
df = pd.DataFrame(all_restaurants).drop_duplicates(subset=["location_id"])
df.reset_index(drop=True, inplace=True)

# Details for each Restaurant: 

restaurant_rows = []

for i, item in df.iterrows():

    loc_id = item["location_id"]
    detail_url = DETAILS_URL.format(loc_id)

    params = {
        "key": API_KEY,
        "language": "en",
        "currency": "EUR"
    }

    detail_res = requests.get(detail_url, headers=headers, params=params)

    if detail_res.status_code != 200:
        print(f"Details error for {loc_id}")
        continue

    details = detail_res.json()

    # Extract details safely
    address = details.get("address_obj", {}) or {}

    restaurant_rows.append({
        "location_id": loc_id,
        "name": details.get("name"),
        "rating": details.get("rating"),
        "num_reviews": details.get("num_reviews"),
        "ranking": details.get("ranking"),
        "price_level": details.get("price_level"),
        "cuisine": ", ".join([c["name"] for c in details.get("cuisine", [])]) if details.get("cuisine") else None,
        "street1": address.get("street1"),
        "city": address.get("city"),
        "country": address.get("country"),
        "postalcode": address.get("postalcode"),
        "address_string": address.get("address_string"),
    })

    time.sleep(0.2)  # Prevent rate limit issues


# Saving Dataframe:

df_restaurants = pd.DataFrame(restaurant_rows)

output_file = "lisbon_restaurants.csv"
df_restaurants.to_csv(output_file, index=False)

print(f"\nDONE! Saved {len(df_restaurants)} restaurants with ratings to {output_file}")


'Lisbon restaurant' returned 10 restaurants
'restaurants Lisbon' returned 10 restaurants
'Lisbon food' returned 10 restaurants
'Lisbon dining' returned 10 restaurants
'Lisbon cafe' returned 10 restaurants
'Lisbon brunch' returned 10 restaurants
'Lisbon lunch' returned 10 restaurants
'Lisbon dinner' returned 10 restaurants
'Lisbon Michelin' returned 10 restaurants
'Lisbon traditional food' returned 10 restaurants
'Lisbon cheap eats' returned 10 restaurants

Total UNIQUE restaurants collected: 62
Fetching rating details for each restaurant...


DONE! Saved 62 restaurants with ratings to lisbon_restaurants.csv


- I got 62 restaurants extracted.

In [80]:
df_restaurants.head(10)

,location_id,name,rating,num_reviews,ranking,price_level,cuisine,street1,city,country,postalcode,address_string
0,12516582,Love Lisbon Restaurant & Bar,4.1,44,None,$$ - $$$,"bar, asian, nepali",R. Jose Antonio Serrano Floor 4,Lisbon,Portugal,1150-033,"R. Jose Antonio Serrano Floor 4, Lisbon 1150-0..."
1,4974326,Madrid Lisbon Restaurant,4.8,4,None,$$ - $$$,"european, spanish, portuguese",325 Lafayette St,Newark,United States,07105-2724,"325 Lafayette St, Newark, NJ 07105-2724"
2,1520116,Floresta Das Escadinhas,4.8,7099,None,$,"mediterranean, barbecue, european, healthy, po...",Rua de Santa Justa N_3,Lisbon,Portugal,1100-483,"Rua de Santa Justa N_3, Lisbon 1100-483 Portugal"
3,2308921,Frade dos Mares,4.8,5546,None,$$ - $$$,"seafood, mediterranean, european, portuguese",Av. Dom Carlos I 55A,Lisbon,Portugal,1200-647,"Av. Dom Carlos I 55A, Lisbon 1200-647 Portugal"
4,1886758,Restaurant Ze da Mouraria,4.3,651,None,$$ - $$$,"mediterranean, european, portuguese",Rua Joao do Outeiro 24,Lisbon,Portugal,1100-292,"Rua Joao do Outeiro 24, Lisbon 1100-292 Portugal"
5,23976748,Restaurant Odaan,4.9,742,None,$,"indian, bar, asian, nepali",Rua dos Cavaleiros 07 LJ 11,Lisbon,Portugal,1100-295,"Rua dos Cavaleiros 07 LJ 11, Lisbon 1100-295 P..."
6,13071752,Jam Club,4.9,1757,None,$,"portuguese, gastropub, dining_bars",Travessa dos Inglesinhos 49,Lisbon,Portugal,1200-223,"Travessa dos Inglesinhos 49, Lisbon 1200-223 P..."
7,15084159,#Treestory Restaurant,4.4,234,None,$$ - $$$,"european, grill, georgian, healthy",Rua Luciano Cordeiro 46A,Lisbon,Portugal,1150-216,"Rua Luciano Cordeiro 46A, Lisbon 1150-216 Port..."
8,1721696,Come Prima Restaurante Italiano,4.8,4357,None,$$ - $$$,"italian, pizza, mediterranean, healthy, neapol...",Rua do Olival 256,Lisbon,Portugal,1200-744,"Rua do Olival 256, Lisbon 1200-744 Portugal"
9,15087513,A Nossa Casa,4.8,1489,None,$$ - $$$,"brazilian, portuguese",Rua da Atalaia No 31,Lisbon,Portugal,1200-037,"Rua da Atalaia No 31, Lisbon 1200-037 Portugal"


## Actractions in Lisbon:

In [81]:
# Load API key
load_dotenv()
API_KEY = os.getenv("API_KEY")

SEARCH_URL = "https://api.content.tripadvisor.com/api/v1/location/search"
DETAILS_URL = "https://api.content.tripadvisor.com/api/v1/location/{}/details"

headers = {"accept": "application/json"}

# Search queries to expand coverage of attractions:
queries = [
    "Lisbon attractions",
    "best things to do Lisbon",
    "Lisbon museum",
    "Lisbon landmark",
    "Lisbon sightseeing",
    "Lisbon tourist spots",
    "Lisbon monuments",
    "Lisbon historical sites",
    "Lisbon outdoors",
    "Lisbon parks",
    "Lisbon viewpoints",
]

all_attractions = []

# Searching for attractions: 

for q in queries:
    url = (
        f"{SEARCH_URL}?key={API_KEY}"
        f"&searchQuery={q}"
        f"&category=attractions"
        f"&language=en"
    )

    response = requests.get(url, headers=headers)
    data = response.json().get("data", [])
    all_attractions.extend(data)

    print(f"'{q}' returned {len(data)} attractions")


# Deduplicate by TripAdvisor Location ID
df = pd.DataFrame(all_attractions).drop_duplicates(subset=["location_id"])
df.reset_index(drop=True, inplace=True)

# Details for each attraction:

attraction_rows = []

for i, item in df.iterrows():

    loc_id = item["location_id"]
    detail_url = DETAILS_URL.format(loc_id)

    params = {
        "key": API_KEY,
        "language": "en",
        "currency": "EUR"
    }

    detail_res = requests.get(detail_url, headers=headers, params=params)
    details = detail_res.json()
    address = details.get("address_obj", {}) or {}

    attraction_rows.append({
        "location_id": loc_id,
        "name": details.get("name"),
        "rating": details.get("rating"),
        "num_reviews": details.get("num_reviews"),
        "ranking": details.get("ranking"),
        "category": details.get("category", {}).get("name") if details.get("category") else None,
        "subcategory": ", ".join(
            [s["name"] for s in details.get("subcategory", [])]
        ) if details.get("subcategory") else None,
        "street1": address.get("street1"),
        "city": address.get("city"),
        "country": address.get("country"),
        "postalcode": address.get("postalcode"),
        "address_string": address.get("address_string"),
    })

    print(f"Retrieved: {details.get('name')} (rating: {details.get('rating')})")

    time.sleep(0.2)  # prevent rate limits

# Saving DataFrame:

df_attractions = pd.DataFrame(attraction_rows)

output_file = "lisbon_attractions_with_ratings.csv"
df_attractions.to_csv(output_file, index=False)

print(f"\nDONE! Saved {len(df_attractions)} attractions with ratings to {output_file}")


'Lisbon attractions' returned 10 attractions
'best things to do Lisbon' returned 10 attractions
'Lisbon museum' returned 10 attractions
'Lisbon landmark' returned 10 attractions
'Lisbon sightseeing' returned 10 attractions
'Lisbon tourist spots' returned 10 attractions
'Lisbon monuments' returned 10 attractions
'Lisbon historical sites' returned 10 attractions
'Lisbon outdoors' returned 10 attractions
'Lisbon parks' returned 10 attractions
'Lisbon viewpoints' returned 10 attractions
Retrieved: Castelo De Sao Jorge (rating: 4.2)
Retrieved: Parque Das Nacoes (rating: 4.3)
Retrieved: Lisbon Cathedral (rating: 4.0)
Retrieved: Praca do Comercio (Terreiro do Paco) (rating: 4.4)
Retrieved: Quake - Museu do Terramoto de Lisboa (rating: 4.6)
Retrieved: Lisbon Falls (rating: 4.2)
Retrieved: Mosteiro dos Jeronimos (rating: 4.5)
Retrieved: Lisboa Story Centre (rating: 4.2)
Retrieved: Oceanario de Lisboa (rating: 4.5)
Retrieved: Arco do Triunfo (rating: 4.4)
Retrieved: Marques Do Pombal (rating: 4.

 - I got 70 attractions extracted.

In [82]:
df_attractions.head(10)

,location_id,name,rating,num_reviews,ranking,category,subcategory,street1,city,country,postalcode,address_string
0,195107,Castelo De Sao Jorge,4.2,30905,None,attraction,"landmarks, attractions",Rua de Santa Cruz do Castelo,Lisbon,Portugal,1100-129,"Rua de Santa Cruz do Castelo, Lisbon 1100-129 ..."
1,546590,Parque Das Nacoes,4.3,4582,None,attraction,"landmarks, attractions, other",Avenida Dom Joao II 13B,Lisbon,Portugal,1990-998,"Avenida Dom Joao II 13B, Lisbon 1990-998 Portugal"
2,23804944,Lisbon Cathedral,4.0,2167,None,attraction,"landmarks, attractions",Largo da Se 1,Lisbon,Portugal,1100-585,"Largo da Se 1, Lisbon 1100-585 Portugal"
3,199878,Praca do Comercio (Terreiro do Paco),4.4,17271,None,attraction,"landmarks, attractions",Avenida Infante Dom Henrique 1C,Lisbon,Portugal,1100-053,"Avenida Infante Dom Henrique 1C, Lisbon 1100-0..."
4,23957745,Quake - Museu do Terramoto de Lisboa,4.6,939,None,attraction,"landmarks, attractions, museums","Rua Cais da Alfandega Velha, 39",Lisbon,Portugal,1300-598,"Rua Cais da Alfandega Velha, 39, Lisbon 1300-5..."
5,2639519,Lisbon Falls,4.2,323,None,attraction,"nature_parks, attractions",None,Graskop,South Africa,1270,Graskop 1270 South Africa
6,195318,Mosteiro dos Jeronimos,4.5,33014,None,attraction,"landmarks, attractions",Praca do Imperio,Lisbon,Portugal,1400-206,"Praca do Imperio, Lisbon 1400-206 Portugal"
7,3928986,Lisboa Story Centre,4.2,932,None,attraction,"museums, attractions","Terreiro do Paco, 78- 81",Lisbon,Portugal,1100-148,"Terreiro do Paco, 78- 81, Lisbon 1100-148 Port..."
8,195144,Oceanario de Lisboa,4.5,40983,None,attraction,"nature_parks, zoos_aquariums, attractions",Esplanada D. Carlos I,Lisbon,Portugal,1990-005,"Esplanada D. Carlos I Doca dos Olivais, Lisbon..."
9,2397769,Arco do Triunfo,4.4,6202,None,attraction,"landmarks, attractions",Rua Augusta 2,Lisbon,Portugal,1100-053,"Rua Augusta 2 Praça do Comércio, Lisbon 1100-0..."


Need to merge all the 3 data frames to start creating the packages.